In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#more import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from pathlib import Path
from sklearn import svm, preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [ ]:
path = Path('/kaggle/input/diamonds/diamonds.csv')
df = pd.read_csv(path)

Lets check head of the DataFrame

In [ ]:
df.head()

In [ ]:
df.info()

all object Dtypes should be converted into numerical or categorical data type. cut, color and clarity are categorical and index column is useless(Unnamed: 0) in this case . And we should check for the null,NaN values first.

In [ ]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
df.isnull().sum(),df.isna().sum()

clarity, from top to bottom tier

IF -> VVS1 -> VVS2 -> VS1 -> VS2 -> SI1 -> SI2 -> I1

diamond color tier list

D -> E -> F -> G -> H -> I -> J

diamond cut tier

Ideal -> Premium -> Very Good -> Good -> Fair

In [ ]:
df.clarity = df.clarity.astype('category')
d1 = dict(enumerate(df.clarity.cat.categories))
d1

In [ ]:
df.cut = df.cut.astype('category')
d2 = dict(enumerate(df.cut.cat.categories))
d2

In [ ]:
df.color = df.color.astype('category')
d3 = dict(enumerate(df.color.cat.categories))
d3

In [ ]:
df.clarity.cat.reorder_categories(['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF'],inplace=True)
df.cut.cat.reorder_categories(['Fair','Good','Very Good','Premium','Ideal'],inplace=True)
df.color.cat.reorder_categories(['J','I','H','G','F','E','D'],inplace=True)

In [ ]:
df.clarity = df.clarity.cat.codes
df.cut = df.cut.cat.codes
df.color = df.color.cat.codes

Now all columns are numerical and we can visualize the data

In [ ]:
correlation = df.corr()
plt.figure(1,figsize=(10,10))
sns.heatmap(correlation,annot=True)    

Price is highly dependent on it's mass(carat) and size(X,Y,Z) ,so let's check first how this parameters are distributed

In [ ]:
sns.distplot(df['carat'] , bins = 20)

In [ ]:
fig = plt.figure(1, figsize = (15,10))
ax = fig.add_subplot(111)
scatter = ax.scatter(df['price'],df['carat'],c=df.clarity)
ax.set_title('price/carat')
ax.set_xlabel('price')
ax.set_ylabel('carat')
plt.colorbar(scatter)

In [ ]:
fig = plt.figure(1, figsize = (15,10))
ax = fig.add_subplot(111)
scatter = ax.scatter(df['price'],df['carat'],c=df.color)
ax.set_title('price/carat')
ax.set_xlabel('price')
ax.set_ylabel('carat')
plt.colorbar(scatter)

In [ ]:
fig = plt.figure(1, figsize = (15,10))
ax = fig.add_subplot(111)
scatter = ax.scatter(df['price'],df['carat'],c=df.cut)
ax.set_title('price/carat')
ax.set_xlabel('price')
ax.set_ylabel('carat')
plt.colorbar(scatter)

Huge diamons are almost never have high clarity, and high clarity still makes huge impact on the price. Same goes with the color. Cut quality also affects the price , making price/carat relationship non-linear.

In [ ]:
sns.countplot(x='clarity',data=df,palette="magma")

In [ ]:
sns.countplot(x='color',data=df,palette="rocket")

In [ ]:
sns.countplot(x='cut',data=df,palette="cubehelix")

high clarity diamons are very rare. Good quality cut and color are common

**FEATURE SCALING**

In [ ]:
scaler = StandardScaler()

In [ ]:
X = scaler.fit_transform(df.drop(['price'],axis=1))
y = scaler.fit_transform(df[['price']])

In [ ]:
df.shape,X.shape,y.shape

train \ test set split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

**creating model** with default settings first . we need metrics as well

In [ ]:
model = KNeighborsRegressor()
model.fit(X_train,y_train)

In [ ]:
pred = model.predict(X_test)

In [ ]:
mse = mean_squared_error(pred,y_test)
rmse = np.sqrt(mean_squared_error(pred,y_test))
mse,rmse

numbers are normalized , to get the idea about price deviation we need unfold price back

In [ ]:
ITpred = scaler.inverse_transform(pred)
ITpred.shape

In [ ]:
ITytest = scaler.inverse_transform(y_test)
ITytest.shape

In [ ]:
ITmse = mean_squared_error(ITpred,ITytest)
ITrmse = np.sqrt(mean_squared_error(ITpred,ITytest))
ITmse,ITrmse

In [ ]:
model.score(X_test,y_test)

**PARAMETER TUNING**

In [ ]:
ptmodel = KNeighborsRegressor()
ptrange = np.arange(1,11)
ptparams = ({'n_neighbors': ptrange})
gridsearch = GridSearchCV(ptmodel,ptparams,cv=10)

In [ ]:
gridsearch.fit(X_train,y_train)
gridsearch.best_params_

re-creating model with best params

In [ ]:
GSmodel = KNeighborsRegressor(n_neighbors=8)
GSmodel.fit(X_train,y_train)

In [ ]:
GSpred = GSmodel.predict(X_test)
GSmse = mean_squared_error(GSpred,y_test)
GSrmse = np.sqrt(mean_squared_error(GSpred,y_test))
GSmse,GSrmse

In [ ]:
GSITpred = scaler.inverse_transform(GSpred)
GSITpred.shape

In [ ]:
GSITytest = scaler.inverse_transform(y_test)
GSITytest.shape

In [ ]:
GSITmse = mean_squared_error(GSITpred,GSITytest)
GSITrmse = np.sqrt(mean_squared_error(GSITpred,GSITytest))
GSITmse,GSITrmse

In [ ]:
GSmodel.score(X_test,y_test)

we've got a bit better results . so, final score

MSE:502017.1463

RMSE:708.5316